In [1]:
%config Completer.use_jedi = False
import warnings

warnings.filterwarnings(action='ignore')

# 스태킹(Stacking)
스태킹은 베이스 학습기와 메타 학습기로 구성되어 있고 베이스 학습기와 메타 학습기는 서포트 벡터 머신, 랜덤 포레스트와 같은 학습 모형이다. 베이스 학습기가 먼저 학습한 후 메타 학습기는 베이스 학습기의 예측을 피쳐 데이터로 활용해 최종 예측을 한다.

스태킹 알고리즘을 활용해 암 여부를 예측하는 모형을 만든다.

In [2]:
# 데이터 불러오기
from sklearn import datasets  # 위스콘신 암 데이터를 사용하기 위해 import 한다.

raw_cancer = datasets.load_breast_cancer()  # 위스콘신 암 데이터를 불러온다.

In [3]:
# 피쳐/타겟 데이터 지정
X = raw_cancer.data  # 위스콘신 암 피쳐 데이터를 저장한다.
y = raw_cancer.target  # 위스콘신 암 타겟 데이터를 저장한다.

In [4]:
# 트레이닝/테스트 데이터 분할
from sklearn.model_selection import train_test_split  # 트레이닝/테스트 데이터 분할을 위해 import 한다.

X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state=0)  # 트레이닝 데이터와 테스트 데이터로 분할한다.
X_tn.shape, X_te.shape

((426, 30), (143, 30))

In [5]:
# 데이터 표준화
from sklearn.preprocessing import StandardScaler  # 데이터 표준화를 위해 import 한다.

std_scale = StandardScaler()  # 표준화 스케일러 객체를 만든다.
# 표준화는 트레이닝 데이터를 기반으로 실행하므로 트레이닝 피쳐 데이터 X_tn을 표준화 스케일러에 적합시킨다.
std_scale.fit(X_tn)
X_tn_std = std_scale.transform(X_tn)  # 트레이닝 피쳐 데이터 X_tn을 표준화 한다.
X_te_std = std_scale.transform(X_te)  # 테스트 피쳐 데이터 X_te를 표준화 한다.

In [6]:
# 데이터 학습
# LogisticRegression - 메타(최종) 학습기, SVC, GaussianNB - 베이스 학습기
from sklearn.linear_model import LogisticRegression  # 로지스틱 회귀 알고리즘을 사용하기 위해 import 한다.
from sklearn.svm import SVC  # 서포트 벡터 머신 알고리즘을 사용하기 위해 import 한다.
from sklearn.naive_bayes import GaussianNB  # 가우시안 나이브 베이즈 알고리즘을 사용하기 위해 import 한다.
# 분류 문제이므로 StackingClassifier를 사용하지만 회귀 문제라면 StackingRegressor를 사용한다.
from sklearn.ensemble import StackingClassifier  # 스태킹 알고리즘을 사용하기 위해 import 한다.

# 베이스 학습기 객체를 만든다.
clf1 = SVC(kernel='linear', random_state=1)  # 서포트 벡터 머신 객체를 만든다.
clf2 = GaussianNB()  # 가우시안 나이브 베이즈 객체를 만든다.

# estimators 옵션으로 베이스 학습기를 지정하고 final_estimator 옵션으로 메타 학습기를 지정해서 스태킹 모델
# 객체를 만든다.
clf_stkg = StackingClassifier(estimators=[('svm', clf1), ('gnb', clf2)], final_estimator=LogisticRegression())
# 표준화된 피쳐 데이터 X_tn_std와 트레이닝 타겟 데이터 y_tn을 넣어서 랜덤 포레스트 알고리즘을 학습시킨다.
clf_stkg.fit(X_tn_std, y_tn)

StackingClassifier(estimators=[('svm', SVC(kernel='linear', random_state=1)),
                               ('gnb', GaussianNB())],
                   final_estimator=LogisticRegression())

In [7]:
# 데이터 예측
pred_stkg = clf_stkg.predict(X_te_std)  # 표준화된 테스트 데이터 X_te_std로 예측한다.
pred_stkg

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0,
       1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1,
       1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0,
       1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0])

In [8]:
# 정확도 평가
from sklearn.metrics import accuracy_score  # 모형 정확도를 평가하기 위해 import 한다.

# accuracy_score() 함수의 인수로 실제 타겟 데이터와 예측된 데이터를 넘겨 정확도를 평가한다.
accuracy = accuracy_score(y_te, pred_stkg)
accuracy

0.965034965034965

In [9]:
# 혼돈 행렬 확인
from sklearn.metrics import confusion_matrix  # 혼돈 행렬을 만들기 위해 import 한다.

# confusion_matrix() 함수에 실제 타겟 데이터와 예측된 데이터를 넘겨 혼돈 행렬을 만든다.
conf_matrix = confusion_matrix(y_te, pred_stkg)
conf_matrix

array([[50,  3],
       [ 2, 88]])

In [10]:
# 분류 리포트 확인
from sklearn.metrics import classification_report  # 분류 리포트를 출력하기 위해 import 한다.

# classification_report() 함수에 실제 타겟 데이터와 예측된 데이터를 넘겨 분류 리포트를 만든다.
class_report = classification_report(y_te, pred_stkg)
print(class_report)

              precision    recall  f1-score   support

           0       0.96      0.94      0.95        53
           1       0.97      0.98      0.97        90

    accuracy                           0.97       143
   macro avg       0.96      0.96      0.96       143
weighted avg       0.96      0.97      0.96       143

